# Module 1, Section 1: Foundational Concepts

In this section, we'll build a simple customer support system for TechHub, a fictional e-commerce electronics store. 

We'll start with the basics:
1. How to make LLM calls with messages
2. How to define tools that access external data (our TechHub database)
3. The manual tool calling loop (involved but educational!)

By the end, you'll understand **why** agent frameworks exist - spoiler, they automate the tedious parts we're about to implement manually

## Setup

First, let's load our environment variables (API keys).


In [1]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

## 1. Basic LLM Call

Let's start simple - just calling an LLM with a message.

**Note:** The default model is set workshop-wide via WORKSHOP_MODEL in the .env file (see config.py).


In [2]:
from langchain.chat_models import init_chat_model
from config import DEFAULT_MODEL

print(f"Using model: {DEFAULT_MODEL}")

# Initialize the model (using workshop default)
llm = init_chat_model(DEFAULT_MODEL)

# Simple string input
response = llm.invoke("What is LangChain in under 10 words?")
response.pretty_print()

Using model: anthropic:claude-haiku-4-5
================================== Ai Message ==================================

Framework for building applications with language models.


## 2. Working with Messages

LLMs work with **messages** that have roles:
- `SystemMessage`: Instructions for how the LLM should behave
- `HumanMessage`: User input
- `AIMessage`: Model responses
- `ToolMessage`: Results from tool execution (we'll see this soon!)


In [3]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Multi-turn conversation with messages
messages = [
    SystemMessage(
        content="You are a helpful customer support assistant for TechHub, an electronics store."
    ),
    HumanMessage(content="Hello!"),
    AIMessage(content="Hi! How can I help you today?"),
    HumanMessage(content="What's the status of order ORD-2024-0123?"),
]

response = llm.invoke(messages)
response.pretty_print()

================================== Ai Message ==================================

Hello! I'd be happy to help you check on your order status.

However, I don't have access to TechHub's order management system, so I'm unable to look up specific order details using order numbers.

To get the status of order **ORD-2024-0123**, here are a few options:

1. **Check your email** - You should have received order confirmation and shipping emails with tracking information
2. **Visit our website** - Log into your account and view your order history
3. **Contact our support team directly** - They can access the system and provide you with real-time updates

Is there anything else I can help you with, such as information about our products, store policies, or general questions?


**Notice the problem:** The LLM can't actually look up the order! It doesn't have access to our database.

This is where **tools** come in.


## 3. Defining Tools

<div align="center">
    <img src="../../static/db_tools.png" alt="Schema Diagram">
</div>

Tools give LLMs the ability to interact with external systems. Let's create three simple tools that query our TechHub database.

The `@tool` decorator automatically:
- Extracts the function signature for the LLM
- Uses the docstring as the tool description
- Handles the input/output formatting


In [4]:
import sqlite3
from pathlib import Path
from langchain.tools import tool

# Path to our TechHub database
DB_PATH = Path("../../data/structured/techhub.db")


@tool
def get_order_details(order_id: str) -> str:
    """Get complete details for a specific order including status, items, and tracking.

    Args:
        order_id: The order ID (e.g., "ORD-2024-0123")

    Returns:
        Formatted string with order status, items, and tracking information.
    """
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    # Get order status and shipping info
    cursor.execute(
        "SELECT status, shipped_date, tracking_number FROM orders WHERE order_id = ?",
        (order_id,),
    )
    order_result = cursor.fetchone()

    if not order_result:
        conn.close()
        return f"No order found with ID: {order_id}"

    status, shipped_date, tracking = order_result

    # Get order items
    cursor.execute(
        """
        SELECT oi.product_id, p.name, oi.quantity, oi.price_per_unit
        FROM order_items oi
        JOIN products p ON oi.product_id = p.product_id
        WHERE oi.order_id = ?
        """,
        (order_id,),
    )
    items_results = cursor.fetchall()
    conn.close()

    # Format response
    response = f"Order {order_id}:\n"
    response += f"  Status: {status}\n"
    response += f"  Shipped: {shipped_date or 'Not yet shipped'}\n"
    response += f"  Tracking: {tracking or 'N/A'}\n"

    if items_results:
        response += "\nItems:\n"
        for product_id, name, quantity, price in items_results:
            response += (
                f"  • {name} (ID: {product_id}) - Qty: {quantity} @ ${price:.2f}\n"
            )

    return response.strip()


@tool
def get_product_price(product_name: str) -> str:
    """Get the current price and availability of a product by name.

    Args:
        product_name: Product name to search for (e.g., "MacBook Air")

    Returns:
        Formatted string with product name, price, and stock status.
    """
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    query = "SELECT name, price, in_stock FROM products WHERE name LIKE ?"
    cursor.execute(query, (f"%{product_name}%",))
    result = cursor.fetchone()
    conn.close()

    if not result:
        return f"No product found matching: {product_name}"

    name, price, in_stock = result
    stock_status = "In Stock" if in_stock else "Out of Stock"
    return f"{name}: ${price:.2f} - {stock_status}"

Let's call of `get_product_price` tool

In [5]:
example_product = "MacBook Air"
result = get_product_price.invoke(example_product)
print(result)

MacBook Air M2 (13-inch, 256GB): $1199.00 - In Stock


Let's inspect how the `@tool` decorator parses the tool information into a `StructuredTool` object.


In [6]:
print("\n--- TOOL NAME ---")
print(get_product_price.name)
print("\n--- TOOL DESCRIPTION ---")
print(get_product_price.description)
print("\n--- TOOL ARGUMENTS ---")
print(get_product_price.args)


--- TOOL NAME ---
get_product_price

--- TOOL DESCRIPTION ---
Get the current price and availability of a product by name.

Args:
    product_name: Product name to search for (e.g., "MacBook Air")

Returns:
    Formatted string with product name, price, and stock status.

--- TOOL ARGUMENTS ---
{'product_name': {'title': 'Product Name', 'type': 'string'}}


## 4. The Manual Tool Calling Loop

Now let's see how LLMs actually use tools! This happens in stages:

1. **Give the LLM access to tools** - Bind tools so the LLM knows what's available
2. **LLM decides when to call a tool** - Based on tool descriptions and the user's query
3. **LLM formats the function call** - Returns which tool to call and with what arguments (but doesn't execute it!)
4. **We manually execute the tool** - Run the actual function to get results
5. **Pass results back to the LLM** - So it can use them to generate a final answer

Let's see each stage in action!


### Step 1: Bind Tools to the Model

First, we tell the LLM what tools are available by "binding" them:


In [7]:
# Bind tools to the model - this tells the LLM what tools are available
tools = [get_order_details, get_product_price]
llm_with_tools = llm.bind_tools(tools)

### Step 2: LLM Decides to Call a Tool

Now let's give the LLM a query. It will decide which tool to call based on the descriptions:


In [8]:
# Ask about an order
query = "What's the status of order ORD-2024-0123?"

messages = [
    SystemMessage(content="You are a helpful customer support assistant."),
    HumanMessage(content=query),
]

# Call the LLM
response = llm_with_tools.invoke(messages)

# Add the AI's tool call to messages
messages.append(response)

response.pretty_print()

================================== Ai Message ==================================

[{'id': 'toolu_01U1XsMjf3dDHezQmxveC33T', 'input': {'order_id': 'ORD-2024-0123'}, 'name': 'get_order_details', 'type': 'tool_use'}]
Tool Calls:
  get_order_details (toolu_01U1XsMjf3dDHezQmxveC33T)
 Call ID: toolu_01U1XsMjf3dDHezQmxveC33T
  Args:
    order_id: ORD-2024-0123


### Step 3: Manually Execute the Tool

The LLM told us WHAT to call and WITH WHAT ARGUMENTS. Now **we** need to actually run the function:


In [13]:
# Extract the tool call information
tool_call = response.tool_calls[0]
tool_name = tool_call["name"]
tool_args = tool_call["args"]

print(f"[Executing]\n {tool_name}(**{tool_args})", "\n")

# Manually execute the tool
if tool_name == "get_order_details":
    tool_result = get_order_details.invoke(tool_args)
elif tool_name == "get_product_price":
    tool_result = get_product_price.invoke(tool_args)

print(f"[Tool Result]\n {tool_result}")

[Executing]
 get_order_details(**{'order_id': 'ORD-2024-0123'}) 

[Tool Result]
 Order ORD-2024-0123:
  Status: Delivered
  Shipped: 2024-12-07
  Tracking: 1Z999AA113527782

Items:
  • JBL Flip 6 Bluetooth Speaker (ID: TECH-AUD-020) - Qty: 1 @ $129.00


### Step 4: Pass Results Back to the LLM

Now we need to send the tool's result back to the LLM so it can generate a final answer for the user. Note that we use a `ToolMessage` type when adding tool result details back to the message history.


In [14]:
from langchain_core.messages import ToolMessage

# Create a ToolMessage with the result
tool_message = ToolMessage(
    content=str(tool_result),
    tool_call_id=tool_call["id"],  # Must match the ID from the tool call
)
messages.append(tool_message)
messages

[SystemMessage(content='You are a helpful customer support assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="What's the status of order ORD-2024-0123?", additional_kwargs={}, response_metadata={}),
 AIMessage(content=[{'id': 'toolu_01U1XsMjf3dDHezQmxveC33T', 'input': {'order_id': 'ORD-2024-0123'}, 'name': 'get_order_details', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01HpDxQVMC5bQWLjyxtaEccz', 'model': 'claude-haiku-4-5-20251001', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 763, 'output_tokens': 66, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-haiku-4-5-20251001', 'model_provider': 'anthropic'}, id='lc_run--fb4e3a19-0429-4fd4-9a6b-691ba12639d4-0', tool_calls=[{'name': 'get_order_details', 'args': {'order_id': 'ORD-20

### Step 5: Get the Final Answer

Now the LLM has the tool result and can give a complete answer to the user:


In [15]:
# Call the LLM again with the tool result
final_response = llm_with_tools.invoke(messages)

final_response.pretty_print()

================================== Ai Message ==================================

Your order **ORD-2024-0123** has been **Delivered**! 

Here are the details:

**Status:** Delivered  
**Shipped:** December 7, 2024  
**Tracking Number:** 1Z999AA113527782

**Item:**
- JBL Flip 6 Bluetooth Speaker (Qty: 1) - $129.00

Your order should have arrived at its destination. If you have any other questions or concerns about this order, feel free to reach out!


### Putting It All Together: The Loop

In real scenarios, the LLM might need to call multiple tools or loop several times. Let's create a function that automates this process:


In [18]:
def run_agent_loop(user_query: str):
    """Run the complete tool calling loop."""

    messages = [
        SystemMessage(
            content="You are a helpful customer support assistant for TechHub."
        ),
        HumanMessage(content=user_query),
    ]

    print(f"\n{'='*60}")
    print(f"User: {user_query}")
    print(f"{'='*60}\n")

    # Keep looping until we get a final answer
    iteration = 0
    while True:
        iteration += 1

        # Call the LLM
        response = llm_with_tools.invoke(messages)

        # Check if LLM wants to use tools
        if not response.tool_calls:
            # No tools needed - we have the final answer!
            print(f"[Final Answer]\n {response.content}\n")
            break

        # LLM wants to use tools
        print(
            f"[Iteration {iteration}] LLM is calling {len(response.tool_calls)} tool(s)..."
        )
        messages.append(response)

        # Execute each tool
        for tool_call in response.tool_calls:
            tool_name = tool_call["name"]
            tool_args = tool_call["args"]

            # Execute the tool
            if tool_name == "get_order_details":
                result = get_order_details.invoke(tool_args)
            elif tool_name == "get_product_price":
                result = get_product_price.invoke(tool_args)

            print(f"  → {tool_name}({list(tool_args.values())[0]}) = {result[:50]}...")

            # Add tool result to messages
            messages.append(
                ToolMessage(content=str(result), tool_call_id=tool_call["id"])
            )

        print()  # Blank line before next iteration

### Try It Out!

Now let's test our agent loop with different queries:


**Example 1: Simple order lookup**


In [19]:
run_agent_loop("What's the status of order ORD-2024-0123?")


User: What's the status of order ORD-2024-0123?

[Iteration 1] LLM is calling 1 tool(s)...
  → get_order_details(ORD-2024-0123) = Order ORD-2024-0123:
  Status: Delivered
  Shipped...

[Final Answer]
 Great! Here's the status of your order **ORD-2024-0123**:

**Status:** ✅ Delivered

**Order Details:**
- **Item:** JBL Flip 6 Bluetooth Speaker
- **Quantity:** 1
- **Price:** $129.00

**Shipping Information:**
- **Shipped Date:** December 7, 2024
- **Tracking Number:** 1Z999AA113527782

Your order has been successfully delivered! If you have any questions about the product or need further assistance, feel free to ask!



**Example 2: Product price lookup**


In [20]:
run_agent_loop("How much does the MacBook Air cost?")


User: How much does the MacBook Air cost?

[Iteration 1] LLM is calling 1 tool(s)...
  → get_product_price(MacBook Air) = MacBook Air M2 (13-inch, 256GB): $1199.00 - In Sto...

[Final Answer]
 The MacBook Air M2 (13-inch, 256GB) costs **$1,199.00** and is currently **in stock** at TechHub.

Is there anything else you'd like to know about this product or any other items?



**Example 3: Multiple tools in one query**


In [21]:
run_agent_loop(
    "What's the status of order ORD-2024-0123 and how much does a MacBook Air cost?"
)


User: What's the status of order ORD-2024-0123 and how much does a MacBook Air cost?

[Iteration 1] LLM is calling 2 tool(s)...
  → get_order_details(ORD-2024-0123) = Order ORD-2024-0123:
  Status: Delivered
  Shipped...
  → get_product_price(MacBook Air) = MacBook Air M2 (13-inch, 256GB): $1199.00 - In Sto...

[Final Answer]
 Great! Here's the information you requested:

**Order ORD-2024-0123 Status:**
- **Status:** Delivered
- **Shipped:** 2024-12-07
- **Tracking Number:** 1Z999AA113527782
- **Item:** JBL Flip 6 Bluetooth Speaker (Qty: 1) - $129.00

**MacBook Air Price:**
- **Product:** MacBook Air M2 (13-inch, 256GB)
- **Price:** $1,199.00
- **Availability:** In Stock

Is there anything else you'd like to know about your order or our products?



## Key Takeaways

**What We Learned**

1. **LLMs can't access external data** without tools
2. **Tools are functions** with clear schemas (name, description, arguments)
3. **The tool calling loop** is: LLM → tool call → execute → result → LLM → repeat until done
4. **This is tedious!** We had to:
   - Manually check for tool calls
   - Execute each tool ourselves
   - Format and append results
   - Manage the loop logic

**Why This Matters**

Understanding the manual loop is crucial because:
- **Every agent framework does this** under the hood
- You can debug agent behavior by understanding what's happening
- You'll appreciate the abstractions in Section 2!

**What's Next**

In **Section 2**, we'll replace all this manual code with `create_agent` - a simple abstraction that:
- Handles the tool calling loop automatically
- Adds memory (conversation history)
- Supports streaming responses
- Requires just a few lines of code!
